# Set up environment and load in data

In [ ]:
library(tidyverse)
library(here)
theme_set(theme_bw())
helpers_path = paste0(here(),'/analysis/helpers/')
source(paste0(helpers_path, '01_clean_behavioral_data.R'))
fig_out_path = paste0(here(), '/outputs/fig/')

Add columns that will be used for plots below.

In [ ]:
data_yn_clean = data_yn_clean %>%
  mutate(correct = ifelse(possiblePayoff>reference & yesChosen == 1, 1, ifelse(possiblePayoff < reference & yesChosen == 0, 1, 0))) %>%
  mutate(type = ifelse(type == 1, "HT", "RE"),
         week = ifelse(week == 1, "Week 1", ifelse(week == 2, "Week 2", ifelse(week == 3, "Week 3", NA))),
         week = factor(week, levels = c("Week 1", "Week 2", "Week 3"))) %>%
  mutate(val_diff = possiblePayoff - reference,
         val_diff_bin = round(val_diff/50),
         val_diff_bin_str = paste0(val_diff_bin*50-25,":",val_diff_bin*50+25),
         val_diff_bin_str = factor(val_diff_bin_str, levels = c("-225:-175", "-175:-125", "-125:-75", "-75:-25", "-25:25", "25:75", "75:125", "125:175", "175:225")))

data_bc_clean = data_bc_clean %>%
  mutate(correct = ifelse(possiblePayoffleft>possiblePayoffright & leftChosen == 1, 1, ifelse(possiblePayoffleft<possiblePayoffright & leftChosen == 0, 1, 0))) %>%
  mutate(type = ifelse(typeLeft == 1, "HT", "RE"),
         week = ifelse(week == 1, "Week 1", ifelse(week == 2, "Week 2", ifelse(week == 3, "Week 3", NA))),
         week = factor(week, levels = c("Week 1", "Week 2", "Week 3"))) %>%
  mutate(val_diff = possiblePayoffleft - possiblePayoffright,
         val_diff_bin = round(val_diff/50),
         val_diff_bin_str = paste0(val_diff_bin*50-25,":",val_diff_bin*50+25),
         val_diff_bin_str = factor(val_diff_bin_str, levels = c("-275:-225","-225:-175", "-175:-125", "-125:-75", "-75:-25", "-25:25", "25:75", "75:125", "125:175", "175:225", "225:275"))) %>%
  filter(!is.na(val_diff_bin_str))

# Accuracy across days

In [ ]:
sub_summary = data_yn_clean %>%
  group_by(day, subnum, type) %>%
  summarise(mean_accuracy = mean(correct),
            sem_accuracy = sd(correct)/sqrt(n()),
            .groups = 'keep') %>%
  mutate(task = "Y/N",
         sum_type = "sub") %>%
  rbind(data_bc_clean %>%
  group_by(day, subnum, type) %>%
  summarise(mean_accuracy = mean(correct),
            sem_accuracy = sd(correct)/sqrt(n()),
            .groups = 'keep') %>%
  mutate(task = "BC",
         sum_type = "sub")) %>%
  mutate(task = factor(task, levels = c("Y/N", "BC")))

group_summary = data_yn_clean %>%
  group_by(day, type) %>%
  summarise(mean_accuracy = mean(correct),
            .groups = 'keep') %>%
  mutate(task = "Y/N",
         sum_type = "group", subnum = 600) %>%
  rbind(data_bc_clean %>%
          group_by(day, type) %>%
          summarise(mean_accuracy = mean(correct),
                    .groups = 'keep') %>%
          mutate(task = "BC",
                 sum_type = "group", subnum = 600)) %>%
  mutate(task = factor(task, levels = c("Y/N", "BC")))

In [ ]:
p = ggplot()+
  geom_point(data = group_summary, aes(day, mean_accuracy, color = type), size = 2)+
  geom_line(data = group_summary, aes(day, mean_accuracy, color = type))+
  geom_point(data = sub_summary, aes(day, mean_accuracy, color = type, shape = as.factor(subnum)), alpha = .3, position=position_dodge(width=0.75))+
  geom_errorbar(data = sub_summary, aes(x = day, color = type, ymin = mean_accuracy - sem_accuracy, ymax = mean_accuracy + sem_accuracy, linetype = as.factor(subnum)), width=0, alpha = .3, position=position_dodge(width=0.75))+
  # geom_line(data = sub_summary, aes(day, mean_accuracy, color = type, linetype = as.factor(subnum)), alpha = .3)+
  facet_wrap(~task)+
  guides(linetype = "none")+
  labs(x = "Day", y = "Accuracy", color="", shape = "")+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_x_continuous(breaks=seq(1,11,1))+
  scale_color_brewer(palette = "Dark2")

# ggsave(file=paste0(fig_out_path, 'ynbc_groupAndSub_accuracyOverDays.jpg'), p, height = 5, width=12, units="in")
p

In [ ]:
rm(sub_summary, group_summary, p)

# Logits across days

## YN Task

In [ ]:
sub_summary = data_yn_clean %>%
  mutate(time_bin = ifelse(day == 1, "Day 1", ifelse(day == 2, "Day 2", ifelse(day == 3, "Day 3", ifelse((day > 3) & (day < 8), "Week 2", "Week 3"))))) %>%
  group_by(subnum, time_bin, val_diff_bin_str, type) %>%
  summarise(mean_y = mean(yesChosen),
            sem_y = sd(yesChosen)/sqrt(n()),
            .groups="keep")

group_summary = data_yn_clean %>%
  mutate(time_bin = ifelse(day == 1, "Day 1", ifelse(day == 2, "Day 2", ifelse(day == 3, "Day 3", ifelse((day > 3) & (day < 8), "Week 2", "Week 3"))))) %>%
  group_by(time_bin, val_diff_bin_str, type) %>%
  summarise(mean_y = mean(yesChosen),
            .groups="keep")

In [ ]:
p = ggplot() +
  geom_hline(data = group_summary, aes(yintercept =.5), color="gray")+
  geom_vline(data = group_summary, aes(xintercept = 5), color="gray")+
  geom_point(data = group_summary, aes(val_diff_bin_str, mean_y, color = type, alpha = time_bin, size = time_bin))+
  geom_line(data = group_summary, aes(val_diff_bin_str, mean_y, color = type, group = time_bin, alpha = time_bin)) +
  facet_wrap(~type)+
  labs(x = "Value Stim - Value Reference", y = "p(Yes)", title="Y/N Task", alpha="", size = "")+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5), size = 12),
        axis.text.y = element_text(size = 12),
        strip.text = element_text(size = 12),
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_color_brewer(palette = "Dark2")+
  scale_x_discrete(labels=c("-225:-175", "", "-125:-75", "", "-25:25", "", "75:125", "", "175:225"))+
  scale_y_continuous(breaks = seq(0,1,.5))+
  guides(linetype = "none", color = "none")+
  scale_alpha_manual(values = c(1, .6, .5, .3, .2))+
  scale_size_manual(values = c(1.5, 2 , 2.5, 3, 3.5))

# ggsave(file=paste0(fig_out_path, 'yn_group_pYesOverValDiffAcrossTime.jpg'), p, height = 4, width = 8, units="in")

p

In [ ]:
p = ggplot() +
  geom_hline(data = sub_summary, aes(yintercept =.5), color="gray")+
  geom_vline(data = sub_summary, aes(xintercept = 5), color="gray")+
  geom_point(data = sub_summary, aes(val_diff_bin_str, mean_y, color = type, alpha = time_bin, size = time_bin))+
  geom_line(data = sub_summary, aes(val_diff_bin_str, mean_y, color = type, group = time_bin, alpha = time_bin)) +
  geom_errorbar(data = sub_summary, aes(x = val_diff_bin_str, ymin = mean_y-sem_y, ymax = mean_y + sem_y, color = type), width = 0, alpha = .6)+
  facet_grid(type~subnum)+
  labs(x = "Value Stim - Value Reference", y = "p(Yes)", title="Y/N Task", alpha="", size = "")+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5)),
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_color_brewer(palette = "Dark2")+
  scale_x_discrete(labels=c("-225:-175", "", "-125:-75", "", "-25:25", "", "75:125", "", "175:225"))+
  scale_y_continuous(breaks = seq(0,1,.5))+
  guides(linetype = "none", color = "none")+
  scale_alpha_manual(values = c(1, .6, .5, .3, .2))+
  scale_size_manual(values = c(1.5, 2 , 2.5, 3, 3.5))

# ggsave(file=paste0(fig_out_path, 'yn_subj_pYesOverValDiffAcrossTime.jpg'), p, height = 5, width = 12, units="in")
p

## BC Task

In [ ]:
sub_summary = data_bc_clean %>%
  mutate(time_bin = ifelse(day == 1, "Day 1", ifelse(day == 2, "Day 2", ifelse(day == 3, "Day 3", ifelse((day > 3) & (day < 8), "Week 2", "Week 3"))))) %>%
  group_by(subnum, time_bin, val_diff_bin_str, type) %>%
  summarise(mean_y = mean(leftChosen),
            sem_y = sd(leftChosen)/sqrt(n()),
            .groups="keep")

group_summary = data_bc_clean %>%
  mutate(time_bin = ifelse(day == 1, "Day 1", ifelse(day == 2, "Day 2", ifelse(day == 3, "Day 3", ifelse((day > 3) & (day < 8), "Week 2", "Week 3"))))) %>%
  group_by(time_bin, val_diff_bin_str, type) %>%
  summarise(mean_y = mean(leftChosen),
            .groups="keep")

In [ ]:
p = ggplot() +
  geom_hline(data = group_summary, aes(yintercept =.5), color="gray")+
  geom_vline(data = group_summary, aes(xintercept = 6), color="gray")+
  geom_point(data = group_summary, aes(val_diff_bin_str, mean_y, color = type, alpha = time_bin, size = time_bin))+
  geom_line(data = group_summary, aes(val_diff_bin_str, mean_y, color = type, group = time_bin, alpha = time_bin)) +
  facet_wrap(~type)+
  labs(x = "Value Left - Value Right", y = "p(Left)", title="BC Task",  alpha="", size = "")+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5), size = 12),
        axis.text.y = element_text(size = 12),
        strip.text = element_text(size = 12),
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_color_brewer(palette = "Dark2")+
  scale_x_discrete(labels=c("","-225:-175", "", "-125:-75", "", "-25:25", "", "75:125", "", "175:225",""))+
  scale_y_continuous(breaks = seq(0,1,.5))+
  guides(linetype = "none", color = "none")+
  scale_alpha_manual(values = c(1, .6, .5, .3, .2))+
  scale_size_manual(values = c(1.5, 2 , 2.5, 3, 3.5))

# ggsave(file=paste0(fig_out_path, 'bc_group_pYesOverValDiffAcrossTime.jpg'), p, height = 4, width = 8, units="in")

p

In [ ]:
p = ggplot() +
  geom_hline(data = sub_summary, aes(yintercept =.5), color="gray")+
  geom_vline(data = sub_summary, aes(xintercept = 6), color="gray")+
  geom_point(data = sub_summary, aes(val_diff_bin_str, mean_y, color = type, alpha = time_bin, size = time_bin))+
  geom_line(data = sub_summary, aes(val_diff_bin_str, mean_y, color = type, group = time_bin, alpha = time_bin)) +
  geom_errorbar(data = sub_summary, aes(x = val_diff_bin_str, ymin = mean_y-sem_y, ymax = mean_y + sem_y, color = type), width = 0, alpha = .6)+
  facet_grid(type~subnum)+
  labs(x = "Value Left - Value Right", y = "p(Left)", title="BC Task", alpha="", size = "")+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5)),
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_color_brewer(palette = "Dark2")+
  scale_x_discrete(labels=c("","-225:-175", "", "-125:-75", "", "-25:25", "", "75:125", "", "175:225",""))+
  scale_y_continuous(breaks = seq(0,1,.5))+
  guides(linetype = "none", color = "none")+
  scale_alpha_manual(values = c(1, .6, .5, .3, .2))+
  scale_size_manual(values = c(1.5, 2 , 2.5, 3, 3.5))

# ggsave(file=paste0(fig_out_path, 'bc_subj_pYesOverValDiffAcrossTime.jpg'), p, height = 5, width = 12, units="in")
p

# RT across days

In [ ]:
sub_summary = data_yn_clean %>%
  group_by(day, subnum, type) %>%
  summarise(mean_y = mean(log(rt)),
            sem_y = sd(log(rt))/sqrt(n()),
            .groups = 'keep') %>%
  mutate(task = "Y/N",
         sum_type = "sub") %>%
  rbind(data_bc_clean %>%
  group_by(day, subnum, type) %>%
  summarise(mean_y = mean(log(rt)),
            sem_y = sd(log(rt))/sqrt(n()),
            .groups = 'keep') %>%
  mutate(task = "BC",
         sum_type = "sub")) %>%
  mutate(task = factor(task, levels = c("Y/N", "BC")))

group_summary = data_yn_clean %>%
  group_by(day, type) %>%
  summarise(mean_y = mean(log(rt)),
            .groups = 'keep') %>%
  mutate(task = "Y/N",
         sum_type = "group", subnum = 600) %>%
  rbind(data_bc_clean %>%
          group_by(day, type) %>%
          summarise(mean_y = mean(log(rt)),
                    .groups = 'keep') %>%
          mutate(task = "BC",
                 sum_type = "group", subnum = 600)) %>%
  mutate(task = factor(task, levels = c("Y/N", "BC")))

In [ ]:
p = ggplot()+
  geom_point(data = group_summary, aes(day, mean_y, color = type), size = 2)+
  geom_line(data = group_summary, aes(day, mean_y, color = type))+
  geom_point(data = sub_summary, aes(day, mean_y, color= type, shape = as.factor(subnum)), alpha = .3, position=position_dodge(width=0.75))+
  geom_errorbar(data = sub_summary, aes(x = day, color = type, ymin = mean_y - sem_y, ymax = mean_y + sem_y, linetype = as.factor(subnum)), width=0, alpha = .3, position=position_dodge(width=0.75))+
  # geom_line(data = sub_summary, aes(day, mean_accuracy, color = type, linetype = as.factor(subnum)), alpha = .3)+
  facet_wrap(~task)+
  guides(linetype = "none")+
  labs(x = "Day", y = "Mean Log RT", color="", shape = "")+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_x_continuous(breaks=seq(1,11,1))+
  scale_color_brewer(palette = "Dark2")

# ggsave(file=paste0(fig_out_path, 'ynbc_groupAndSub_meanLogRTOverDays.jpg'), p, height = 5, width=12, units="in")
p

# RT slope across days

## YN Task

In [ ]:
data_yn_clean = data_yn_clean %>%
  mutate(abs_val_diff_bin_str =  paste0(abs(val_diff_bin)*50-25,":",abs(val_diff_bin)*50+25),
         abs_val_diff_bin_str = ifelse(abs_val_diff_bin_str ==  "-25:25", "0:25", abs_val_diff_bin_str),
         abs_val_diff_bin_str = factor(abs_val_diff_bin_str, levels = c("0:25", "25:75", "75:125", "125:175", "175:225", "225:275")))

In [ ]:
sub_summary = data_yn_clean %>%
  mutate(time_bin = ifelse(day == 1, "Day 1", ifelse(day == 2, "Day 2", ifelse(day == 3, "Day 3", ifelse((day > 3) & (day < 8), "Week 2", "Week 3"))))) %>%
  group_by(subnum, time_bin, abs_val_diff_bin_str, type) %>%
  summarise(mean_y = mean(log(rt)),
            sem_y = sd(log(rt))/sqrt(n()),
            .groups="keep")

group_summary = data_yn_clean %>%
  mutate(time_bin = ifelse(day == 1, "Day 1", ifelse(day == 2, "Day 2", ifelse(day == 3, "Day 3", ifelse((day > 3) & (day < 8), "Week 2", "Week 3"))))) %>%
  group_by(time_bin, abs_val_diff_bin_str, type) %>%
  summarise(mean_y = mean(log(rt)),
            .groups="keep")

In [ ]:
p = ggplot() +
  geom_point(data = group_summary, aes(abs_val_diff_bin_str, mean_y, color = type, alpha = time_bin, size = time_bin))+
  geom_line(data = group_summary, aes(abs_val_diff_bin_str, mean_y, color = type, group = time_bin, alpha = time_bin)) +
  facet_wrap(~type)+
  labs(x = "|Value Stim - Value Reference|", y = "Mean Log RT", title="Y/N Task", alpha="", size = "")+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5), size = 12),
        axis.text.y = element_text(size = 12),
        strip.text = element_text(size = 12),
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_color_brewer(palette = "Dark2")+
  scale_y_continuous(breaks = seq(0,1,.5))+
  guides(linetype = "none", color = "none")+
  scale_alpha_manual(values = c(1, .6, .5, .3, .2))+
  scale_size_manual(values = c(1.5, 2 , 2.5, 3, 3.5))

# ggsave(file=paste0(fig_out_path, 'yn_group_MeanLogRTOverValDiffAcrossTime.jpg'), p, height = 4, width = 8, units="in")

p

In [ ]:
p = ggplot() +
   geom_point(data = sub_summary, aes(abs_val_diff_bin_str, mean_y, color = type, alpha = time_bin, size = time_bin))+
  geom_line(data = sub_summary, aes(abs_val_diff_bin_str, mean_y, color = type, group = time_bin, alpha = time_bin)) +
  geom_errorbar(data = sub_summary, aes(x = abs_val_diff_bin_str, ymin = mean_y-sem_y, ymax = mean_y + sem_y, color = type), width = 0, alpha = .6)+
  facet_grid(type~subnum)+
  labs(x = "|Value Stim - Value Reference|", y = "Mean Log RT", title="Y/N Task", alpha="", size = "")+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5)),
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_color_brewer(palette = "Dark2")+
  scale_y_continuous(breaks = seq(0,1,.5))+
  guides(linetype = "none", color = "none")+
  scale_alpha_manual(values = c(1, .6, .5, .3, .2))+
  scale_size_manual(values = c(1.5, 2 , 2.5, 3, 3.5))

# ggsave(file=paste0(fig_out_path, 'yn_subj_MeanLogRTValDiffAcrossTime.jpg'), p, height = 5, width = 12, units="in")
p

## BC Task

In [ ]:
data_bc_clean = data_bc_clean %>%
  mutate(abs_val_diff_bin_str =  paste0(abs(val_diff_bin)*50-25,":",abs(val_diff_bin)*50+25),
         abs_val_diff_bin_str = ifelse(abs_val_diff_bin_str ==  "-25:25", "0:25", abs_val_diff_bin_str),
         abs_val_diff_bin_str = factor(abs_val_diff_bin_str, levels = c("0:25", "25:75", "75:125", "125:175", "175:225", "225:275")))

In [ ]:
sub_summary = data_bc_clean %>%
  mutate(time_bin = ifelse(day == 1, "Day 1", ifelse(day == 2, "Day 2", ifelse(day == 3, "Day 3", ifelse((day > 3) & (day < 8), "Week 2", "Week 3"))))) %>%
  group_by(subnum, time_bin, abs_val_diff_bin_str, type) %>%
  summarise(mean_y = mean(log(rt)),
            sem_y = sd(log(rt))/sqrt(n()),
            .groups="keep")

group_summary = data_bc_clean %>%
  mutate(time_bin = ifelse(day == 1, "Day 1", ifelse(day == 2, "Day 2", ifelse(day == 3, "Day 3", ifelse((day > 3) & (day < 8), "Week 2", "Week 3"))))) %>%
  group_by(time_bin, abs_val_diff_bin_str, type) %>%
  summarise(mean_y = mean(log(rt)),
            .groups="keep")

In [ ]:
p = ggplot() +
  geom_point(data = group_summary, aes(abs_val_diff_bin_str, mean_y, color = type, alpha = time_bin, size = time_bin))+
  geom_line(data = group_summary, aes(abs_val_diff_bin_str, mean_y, color = type, group = time_bin, alpha = time_bin)) +
  facet_wrap(~type)+
  labs(x = "|Value Stim - Value Reference|", y = "Mean Log RT", title="BC Task", alpha="", size = "")+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5), size = 12),
        axis.text.y = element_text(size = 12),
        strip.text = element_text(size = 12),
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_color_brewer(palette = "Dark2")+
  scale_y_continuous(breaks = seq(0,1,.5))+
  guides(linetype = "none", color = "none")+
  scale_alpha_manual(values = c(1, .6, .5, .3, .2))+
  scale_size_manual(values = c(1.5, 2 , 2.5, 3, 3.5))

# ggsave(file=paste0(fig_out_path, 'bc_group_MeanLogRTOverValDiffAcrossTime.jpg'), p, height = 4, width = 8, units="in")

p

In [ ]:
p = ggplot() +
  geom_point(data = sub_summary, aes(abs_val_diff_bin_str, mean_y, color = type, shape = time_bin), size = 1.5)+
  geom_line(data = sub_summary, aes(abs_val_diff_bin_str, mean_y, color = type, group = time_bin, linetype = time_bin)) +
  geom_errorbar(data = sub_summary, aes(x = abs_val_diff_bin_str, ymin = mean_y-sem_y, ymax = mean_y + sem_y, color = type), width = 0, alpha = .6)+
  facet_grid(type~subnum)+
  labs(x = "|Value Stim - Value Reference|", y = "Mean Log RT", title="BC Task", color="", shape="")+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5)),
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_color_brewer(palette = "Dark2")+
  scale_y_continuous(breaks = seq(0,1,.5))+
  guides(linetype = "none")

# ggsave(file=paste0(fig_out_path, 'bc_subj_MeanLogRTValDiffAcrossTime.jpg'), p, height = 5, width = 12, units="in")
p

# Fixation durations

In [ ]:
data_bc_fix = read.csv(paste0(here(), '/inputs/data_fixBC.csv'))

data_bc_fix = data_bc_fix %>%
  mutate(correct = ifelse(possiblePayoffleft>possiblePayoffright & leftChosen == 1, 1, ifelse(possiblePayoffleft<possiblePayoffright & leftChosen == 0, 1, 0))) %>%
  mutate(type = ifelse(typeLeft == 1, "HT", "RE")) %>%
  mutate(val_diff = possiblePayoffleft - possiblePayoffright,
         val_diff_bin = round(val_diff/50),
         val_diff_bin_str = paste0(val_diff_bin*50-25,":",val_diff_bin*50+25),
         val_diff_bin_str = factor(val_diff_bin_str, levels = c("-275:-225","-225:-175", "-175:-125", "-125:-75", "-75:-25", "-25:25", "25:75", "75:125", "125:175", "175:225", "225:275")),
         abs_val_diff_bin_str =  paste0(abs(val_diff_bin)*50-25,":",abs(val_diff_bin)*50+25),
         abs_val_diff_bin_str = ifelse(abs_val_diff_bin_str ==  "-25:25", "0:25", abs_val_diff_bin_str),
         abs_val_diff_bin_str = factor(abs_val_diff_bin_str, levels = c("0:25", "25:75", "75:125", "125:175", "175:225", "225:275"))) %>%
  filter(!is.na(val_diff_bin_str))

In [ ]:
sub_summary = data_bc_fix %>%
  select(subnum, day, type, firstFix, middleFix, lastFix, fixDuration) %>%
  mutate(fixType = ifelse(firstFix == 1, "first", ifelse(middleFix == 1, "middle", ifelse(lastFix == 1, "last", NA)))) %>%
  group_by(subnum, day, type, fixType) %>%
  summarise(mean_duration = mean(fixDuration),
            sem_duration = sd(fixDuration)/sqrt(n()),
            .groups = "keep") %>%
  mutate(day = paste0("Day ", day),
         day = factor(day, levels = c("Day 3", "Day 7", "Day 11")),
         fixType = factor(fixType, levels = c("first", "middle", "last")))

group_summary = data_bc_fix %>%
  select(subnum, day, type, firstFix, middleFix, lastFix, fixDuration) %>%
  mutate(fixType = ifelse(firstFix == 1, "first", ifelse(middleFix == 1, "middle", ifelse(lastFix == 1, "last", NA)))) %>%
  group_by(day, type, fixType) %>%
  summarise(mean_duration = mean(fixDuration),
            .groups = "keep") %>%
  mutate(day = paste0("Day ", day),
         day = factor(day, levels = c("Day 3", "Day 7", "Day 11")),
         fixType = factor(fixType, levels = c("first", "middle", "last")))

In [ ]:
p = ggplot()+
    geom_point(data = sub_summary, aes(fixType, mean_duration, color= type, shape = as.factor(subnum)), alpha = 1, size=1.5)+
  geom_line(data = sub_summary, aes(fixType, mean_duration, color= type, group = as.factor(subnum), linetype = as.factor(subnum)), alpha = .4)+
  geom_errorbar(data = sub_summary, aes(x = fixType, color = type, ymin = mean_duration - sem_duration, ymax =  mean_duration + sem_duration, linetype = as.factor(subnum)), width=0, alpha = .5)+
  labs(title = "BC Task", x = "Fixation Type", y = "Duration (ms)", color="", shape = "")+
  facet_grid(type ~ day)+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(margin = margin(t = 5)),
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_color_brewer(palette = "Dark2")+
  guides(linetype = "none")

# ggsave(file=paste0(fig_out_path, 'bc_subj_fixDurOverFixType.jpg'), p, height = 5, width = 8, units="in")
p  

# Number of fixations

In [ ]:
sub_summary = data_bc_fix %>%
  filter(lastFix != 1) %>%
  group_by(subnum, day, type, val_diff_bin_str) %>%
  summarise(mean_num_fix = mean(numTotalFix), 
            sem_num_fix = sd(numTotalFix)/sqrt(n()),
            .groups="keep") %>%
  mutate(day = paste0("Day ", day),
         day = factor(day, levels = c("Day 3", "Day 7", "Day 11")))

In [ ]:
p = sub_summary %>%
  ggplot(aes(val_diff_bin_str, mean_num_fix, color = type))+
  geom_vline(aes(xintercept = 6), color="gray")+
  geom_point(size = 1.5)+
  geom_errorbar(aes(ymin = mean_num_fix - sem_num_fix, ymax = mean_num_fix + sem_num_fix), width = 0, size = 1, alpha = .5)+
  geom_line(aes(group = type))+
  labs(title = "BC Task", x = "Value Left - Value Right", y = "Mean Number of Fixations", color="")+
  # facet_grid(type ~ day)+
  facet_grid(subnum ~ day, scales = 'free_y')+
  theme(panel.grid = element_blank(),
        legend.position = "none",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5)),
        legend.box.margin=margin(t = -10))+
  scale_color_brewer(palette = "Dark2")+
  scale_y_continuous(breaks = seq(0,10,2))+
  scale_x_discrete(labels=c("","-225:-175", "", "-125:-75", "", "-25:25", "", "75:125", "", "175:225", ""))

# ggsave(file=paste0(fig_out_path, 'bc_subj_meanNumFixOverValDiff.jpg'), p, height = 6, width = 8, units="in")
p

In [ ]:
sub_summary = data_bc_fix %>%
  filter(lastFix != 1) %>%
  group_by(subnum, day, type,abs_val_diff_bin_str) %>%
  summarise(mean_num_fix = mean(numTotalFix), 
            sem_num_fix = sd(numTotalFix)/sqrt(n()),
            .groups="keep") %>%
  mutate(day = paste0("Day ", day),
         day = factor(day, levels = c("Day 3", "Day 7", "Day 11")))

In [ ]:
p = sub_summary %>%
  ggplot(aes(abs_val_diff_bin_str, mean_num_fix, color = type))+
  geom_point(size = 1.5)+
  geom_errorbar(aes(ymin = mean_num_fix - sem_num_fix, ymax = mean_num_fix + sem_num_fix), width = 0, size = 1, alpha = .5)+
  geom_line(aes(group = type))+
  labs(title = "BC Task", x = "|Value Left - Value Right|", y = "Mean Number of Fixations", color="")+
  facet_grid(subnum ~ day, scales = 'free_y')+
  theme(panel.grid = element_blank(),
        legend.position = "none",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5)),
        legend.box.margin=margin(t = -10))+
  scale_color_brewer(palette = "Dark2")+
  scale_y_continuous(breaks = seq(0,10,2))

# ggsave(file=paste0(fig_out_path, 'bc_subj_meanNumFixOverAbsValDiff.jpg'), p, height = 6, width = 8, units="in")
p

# First Fixation Location

In [ ]:
p = data_bc_fix %>%
  filter(firstFix == 1) %>%
  group_by(subnum, day, type, val_diff_bin_str) %>%
  summarise(mean_first_fix_left = mean(leftFix),
            sem_first_fix_left = sd(leftFix)/sqrt(n()),
            num_trials = n(), .groups="keep") %>%
  mutate(day = paste0("Day ", day),
         day = factor(day, levels = c("Day 3", "Day 7", "Day 11"))) %>%
  ggplot(aes(val_diff_bin_str, mean_first_fix_left, color = type))+
   geom_hline(aes(yintercept=.5), color="gray")+
  geom_vline(aes(xintercept=6), color="gray")+
  geom_point(size = 1.5)+
  geom_errorbar(aes(ymin = mean_first_fix_left - sem_first_fix_left, ymax = mean_first_fix_left + sem_first_fix_left), width = 0, size = 1, alpha = .6)+
  geom_line(aes(group = type))+
  labs(title = "BC Task", x = "Value Left - Value Right", y = "p(First Fixation = Left)", color="")+
  facet_grid(subnum ~ day)+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5)),
        legend.box.margin=margin(t = -10))+
  scale_color_brewer(palette = "Dark2", guide = "none")+
  scale_x_discrete(labels=c("","-225:-175", "", "-125:-75", "", "-25:25", "", "75:125", "", "175:225", ""))+
  scale_y_continuous(breaks = seq(0,1,.5))

ggsave(file=paste0(fig_out_path, 'bc_subj_pFirstFixLeftOverValDiff.jpg'), p, height = 6, width=8, units="in")
p

# First fixation effect

In [ ]:
p = data_bc_fix %>%
  filter(!is.na(val_diff_bin_str)) %>%
  filter(firstFix == 1) %>%
  group_by(subnum, day, val_diff_bin_str, leftFix, type) %>%
  summarise(.groups = 'keep',
            mean_left_choice = mean(leftChosen),
            sem_left_choice = sd(leftChosen)/sqrt(n())) %>%
  ungroup() %>%
  mutate(lastFixLeft = ifelse(leftFix == 1, "Left", "Right")) %>%
  select(-leftFix) %>%
  mutate(day = paste0("Day ", day),
         day = factor(day, levels = c("Day 3", "Day 7", "Day 11")),
         sem_left_choice = ifelse(is.na(sem_left_choice), 0, sem_left_choice)) %>%
  filter(type == "HT") %>%
  ggplot(aes(val_diff_bin_str, mean_left_choice, color = type))+
  geom_hline(aes(yintercept=.5), color="gray")+
  geom_vline(aes(xintercept=6), color="gray")+
  geom_point(aes(shape = lastFixLeft), size=1.5)+
  geom_errorbar(aes(ymin = mean_left_choice - sem_left_choice, ymax = mean_left_choice + sem_left_choice), width = 0, size = 1, alpha = .6)+
  geom_line(aes(group = lastFixLeft, linetype = lastFixLeft))+
  labs(title = "BC Task", x = "Value Left - Value Right", y = "p(Left Choice)", shape="First Fixation")+
  facet_grid(subnum ~ day)+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5)),
        legend.box.margin=margin(t = -10))+
  scale_x_discrete(labels=c("","-225:-175", "", "-125:-75", "", "-25:25", "", "75:125", "", "175:225", ""))+
  scale_y_continuous(breaks = seq(0,1,.5))+
  scale_color_brewer(palette = "Dark2", guide = "none")+
  scale_linetype(guide = "none")

# ggsave(file=paste0(fig_out_path, 'bc_subj_pLeftOverValDiffByFirstFixByStim.jpg'), p, height = 6, width=8, units="in")
p

In [ ]:
p = data_bc_fix %>%
  filter(!is.na(val_diff_bin_str)) %>%
  filter(firstFix == 1) %>%
  group_by(subnum, day, val_diff_bin_str, leftFix, type) %>%
  summarise(.groups = 'keep',
            mean_left_choice = mean(leftChosen),
            sem_left_choice = sd(leftChosen)/sqrt(n())) %>%
  ungroup() %>%
  mutate(lastFixLeft = ifelse(leftFix == 1, "Left", "Right")) %>%
  select(-leftFix) %>%
  mutate(day = paste0("Day ", day),
         day = factor(day, levels = c("Day 3", "Day 7", "Day 11")),
         sem_left_choice = ifelse(is.na(sem_left_choice), 0, sem_left_choice)) %>%
  filter(type == "RE") %>%
  ggplot(aes(val_diff_bin_str, mean_left_choice))+
  geom_hline(aes(yintercept=.5), color="gray")+
  geom_vline(aes(xintercept=6), color="gray")+
  geom_point(aes(shape = lastFixLeft), size=1.5, color = "#D95F02")+
  geom_errorbar(aes(ymin = mean_left_choice - sem_left_choice, ymax = mean_left_choice + sem_left_choice), width = 0, size = 1, alpha = .6, color = "#D95F02")+
  geom_line(aes(group = lastFixLeft, linetype = lastFixLeft), color = "#D95F02")+
  labs(title = "BC Task", x = "Value Left - Value Right", y = "p(Left Choice)", shape="First Fixation")+
  facet_grid(subnum ~ day)+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5)),
        legend.box.margin=margin(t = -10))+
  scale_x_discrete(labels=c("","-225:-175", "", "-125:-75", "", "-25:25", "", "75:125", "", "175:225", ""))+
  scale_y_continuous(breaks = seq(0,1,.5))+
  scale_color_brewer(palette = "Dark2", guide = "none")+
  scale_linetype(guide = "none")

# ggsave(file=paste0(fig_out_path, 'bc_subj_pLeftOverValDiffByFirstFixByStim.jpg'), p, height = 6, width=8, units="in")
p

# Last fixation effect

In [ ]:
p = data_bc_fix %>%
  filter(!is.na(val_diff_bin_str)) %>%
  filter(lastFix == 1) %>%
  group_by(subnum, day, val_diff_bin_str, leftFix, type) %>%
  summarise(.groups = 'keep',
            mean_left_choice = mean(leftChosen),
            sem_left_choice = sd(leftChosen)/sqrt(n())) %>%
  ungroup() %>%
  mutate(lastFixLeft = ifelse(leftFix == 1, "Left", "Right")) %>%
  select(-leftFix) %>%
  mutate(day = paste0("Day ", day),
         day = factor(day, levels = c("Day 3", "Day 7", "Day 11")),
         sem_left_choice = ifelse(is.na(sem_left_choice), 0, sem_left_choice)) %>%
  filter(type == "HT") %>%
  ggplot(aes(val_diff_bin_str, mean_left_choice, color = type))+
  geom_hline(aes(yintercept=.5), color="gray")+
  geom_vline(aes(xintercept=6), color="gray")+
  geom_point(aes(shape = lastFixLeft), size=2.5)+
  geom_errorbar(aes(ymin = mean_left_choice - sem_left_choice, ymax = mean_left_choice + sem_left_choice), width = 0, size=1)+
  geom_line(aes(group = lastFixLeft, linetype = lastFixLeft))+
  labs(title = "BC Task", x = "Value Left - Value Right", y = "p(Left Choice)", shape="Last Fixation")+
  facet_grid(subnum ~ day)+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5)),
        legend.box.margin=margin(t = -10))+
  scale_x_discrete(labels=c("","-225:-175", "", "-125:-75", "", "-25:25", "", "75:125", "", "175:225", ""))+
  scale_y_continuous(breaks = seq(0,1,.5))+
  scale_color_brewer(palette = "Dark2", guide = "none")+
  scale_linetype(guide = "none")

# ggsave(file=paste0(fig_out_path, 'bc_subj_pLeftOverValDiffByLastFixByStim.jpg'), p, height = 6, width=8, units="in")
p

In [ ]:
p = data_bc_fix %>%
  filter(!is.na(val_diff_bin_str)) %>%
  filter(lastFix == 1) %>%
  group_by(subnum, day, val_diff_bin_str, leftFix, type) %>%
  summarise(.groups = 'keep',
            mean_left_choice = mean(leftChosen),
            sem_left_choice = sd(leftChosen)/sqrt(n())) %>%
  ungroup() %>%
  mutate(lastFixLeft = ifelse(leftFix == 1, "Left", "Right")) %>%
  select(-leftFix) %>%
  mutate(day = paste0("Day ", day),
         day = factor(day, levels = c("Day 3", "Day 7", "Day 11")),
         sem_left_choice = ifelse(is.na(sem_left_choice), 0, sem_left_choice)) %>%
  filter(type == "RE") %>%
  ggplot(aes(val_diff_bin_str, mean_left_choice))+
  geom_hline(aes(yintercept=.5), color="gray")+
  geom_vline(aes(xintercept=6), color="gray")+
  geom_point(aes(shape = lastFixLeft), size=2.5, color = "#D95F02")+
  geom_errorbar(aes(ymin = mean_left_choice - sem_left_choice, ymax = mean_left_choice + sem_left_choice), width = 0, size=1,  color = "#D95F02")+
  geom_line(aes(group = lastFixLeft, linetype = lastFixLeft),  color = "#D95F02")+
  labs(title = "BC Task", x = "Value Left - Value Right", y = "p(Left Choice)", shape="Last Fixation")+
  facet_grid(subnum ~ day)+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5)),
        legend.box.margin=margin(t = -10))+
  scale_x_discrete(labels=c("","-225:-175", "", "-125:-75", "", "-25:25", "", "75:125", "", "175:225", ""))+
  scale_y_continuous(breaks = seq(0,1,.5))+
  # scale_color_brewer(palette = "Dark2", guide = "none")+
  scale_linetype(guide = "none")

# ggsave(file=paste0(fig_out_path, 'bc_subj_pLeftOverValDiffByLastFixByStim.jpg'), p, height = 6, width=8, units="in")
p

In [ ]:
rm(p, sub_summary, group_summary)